# Data Pre-processing steps

In [13]:
# _importing required libraries
import pandas as pd
import os
import glob

In [14]:
# _file paths of input and output data
subject_id = 'subject104'
input_file_path = os.getcwd() + f'/../data/input_dat/{subject_id}.dat'
output_file_path = os.getcwd() + f'/../data/output_csv/{subject_id}_processed.csv'

In [15]:
# _giving column names considered columns from dataset are
# timestamp (s)
# activityID  
# heart rate (bpm)
# (IMU hand) 3D-acceleration data (ms-2), scale: ±16g, resolution: 13-bit 
# (IMU hand) 3D-gyroscope data (rad/s)
col_names = ['timestamp (s)', 'activityID', 'heart rate (bpm)', 'X1', 'Y1', 'Z1', 'X2', 'Y2', 'Z2']
required_cols = [0,1,2,4,5,6,10,11,12]
# _loading data into pandas dataframe
main_df = pd.read_csv(input_file_path, header=None, names=col_names, sep='\s+', usecols=required_cols, engine='python')
# _printing few records of data
main_df

,timestamp (s),activityID,heart rate (bpm),X1,Y1,Z1,X2,Y2,Z2
0,5.71,0,NaN,-1.54309,5.78581,7.73342,-0.028974,0.047977,-0.012797
1,5.72,0,NaN,-1.69986,5.55834,7.61638,0.000358,0.026022,-0.008945
2,5.73,0,NaN,-1.54603,5.55784,7.69566,0.017085,0.011208,-0.040957
3,5.74,0,NaN,-1.57954,5.59708,7.81049,0.013649,0.008197,0.003188
4,5.75,0,NaN,-1.51225,5.55654,7.58071,-0.014763,-0.006078,0.019068
...,...,...,...,...,...,...,...,...,...
329571,3301.42,0,NaN,-4.46502,6.40756,-5.86000,0.004286,0.056136,0.018818
329572,3301.43,0,NaN,-4.62099,6.29390,-5.97741,0.011409,0.015436,0.024282
329573,3301.44,0,NaN,-4.69015,6.44758,-5.82502,0.000941,0.028801,0.000344
329574,3301.45,0,NaN,-4.53525,6.40948,-5.70711,-0.013739,0.048928,0.011665


In [16]:
# _showing null values count for each column
main_df.isna().sum()

timestamp (s)            0
activityID               0
heart rate (bpm)    299461
X1                    2214
Y1                    2214
Z1                    2214
X2                    2214
Y2                    2214
Z2                    2214
dtype: int64

In [17]:
# _dropping heart rate column
del main_df['heart rate (bpm)']
# _dropping rows which has all NaN values in all sensor fields
main_df.dropna(how='all', subset= ['X1', 'Y1', 'Z1', 'X2', 'Y2', 'Z2'], inplace= True)
main_df[pd.isnull(main_df).any(axis=1)]

,timestamp (s),activityID,X1,Y1,Z1,X2,Y2,Z2


In [18]:
# _showing null values count for each column
main_df.isna().sum()

timestamp (s)    0
activityID       0
X1               0
Y1               0
Z1               0
X2               0
Y2               0
Z2               0
dtype: int64

In [19]:
# _columns list for which we need to perform ouliers operation
cols = ['X1', 'Y1', 'Z1', 'X2', 'Y2', 'Z2']
# _calculating threshold values to remove outliers which are +3 or -3 std away from mean
lower = main_df[cols].quantile(0.01)
higher  = main_df[cols].quantile(0.99)
print(lower,higher)

X1   -14.859546
Y1    -3.811070
Z1    -6.666607
X2    -3.368499
Y2    -2.003013
Z2    -2.728665
Name: 0.01, dtype: float64 X1     5.637650
Y1    11.699456
Z1    10.522617
X2     3.638149
Y2     2.029964
Z2     2.948515
Name: 0.99, dtype: float64


In [20]:
#_checking and removing outliers
main_df = main_df[((main_df[cols] < higher) & (main_df[cols] > lower)).any(axis=1)]

In [21]:
# _activityID 0 represents trasient activities which are not useful in this project.
# _droping the rows whose activityID column is 0
# _considering only three activities for basic testing
main_df = main_df[main_df['activityID'].isin([1,2,3])] 
main_df.groupby(['activityID']).agg(['count'])

,timestamp (s),X1,Y1,Z1,X2,Y2,Z2
,count,count,count,count,count,count,count
activityID,,,,,,,
1,23045,23045,23045,23045,23045,23045,23045
2,25482,25482,25482,25482,25482,25482,25482
3,24683,24683,24683,24683,24683,24683,24683


In [22]:
# _write data to ouput file
main_df.to_csv(output_file_path, encoding='utf-8', index=False, header=True)

# Merging output csv files

In [26]:
def merge_csv_files(input_csv_file_path, merged_csv_file_path):
    
    # _get all csv files inside file_path
    csv_files = glob.glob(input_csv_file_path)

    df_list = []
    for csv in csv_files:
        df_list.append(pd.read_csv(csv))

    # _stack all the csv files into a single file
    result = pd.concat(df_list)
    
    # _write data to ouput file
    result.to_csv(merged_csv_file_path, encoding='utf-8', index=False, header=False)

In [25]:
# _file paths of input and output data
input_csv_file_path = os.getcwd() + f'/../data/output_csv/*.csv'
merged_csv_file_path = os.getcwd() + f'/../data/processed_data.csv'
#merge_csv_files(input_csv_file_path, merged_csv_file_path)